# **Computational Theory Assessment (2024/2025)**
Ronan Francis (G00403092)
Computational Theory

## **Table of Contents**
1. [Task 1: Binary Representations](#task-1-binary-representations)
2. [Task 2: Hash Functions](#task-2-hash-functions)
3. [Task 3: SHA256 Padding](#task-3-sha256-padding)
4. [Task 4: Prime Numbers](#task-4-prime-numbers)
5. [Task 5: Roots](#task-5-roots)
6. [Task 6: Proof of Work](#task-6-proof-of-work)
7. [Task 7: Turing Machines](#task-7-turing-machines)
8. [Task 8: Computational Complexity](#task-8-computational-complexity)

## **Task 1: Binary Representations**

In this task, we focus on implementing several core functions that manipulate binary representations of 32-bit unsigned integers. These operations are crucial in many fields, including cryptography (like SHA-256), data encoding, and low-level programming.

### Circular Shifts

Circular shifts (or bit rotations) involve shifting the bits of a number to the left or right, with the bits that "fall off" one end reappearing on the opposite end. This operation is distinct from logical shifts, where bits shifted off one end are simply discarded and zeros are introduced on the other end. Circular shifts maintain the bit-length and the overall "weight" of the data.

For example, when you perform a left circular shift (`rotl`) on a 32-bit integer:
- The bits are moved to the left by a specified number of positions.
- The bits that move past the most significant bit are wrapped around to the least significant bit positions.

Similarly, a right circular shift (`rotr`) moves bits to the right with wrapping from the rightmost bits to the leftmost positions.

For more detailed information, see the [Circular Shift Wikipedia article](https://en.wikipedia.org/wiki/Circular_shift).

### Bitwise Operators in Python

Python provides several bitwise operators that allow for efficient manipulation of integers at the binary level:

- **AND (`&`):** Compares each bit of two integers and returns `1` only if both bits are `1`.
- **OR (`|`):** Compares each bit of two integers and returns `1` if at least one of the bits is `1`.
- **XOR (`^`):** Returns `1` for each bit position where the corresponding bits of the two operands are different.
- **NOT (`~`):** Inverts the bits of the operand.
- **Left Shift (`<<`):** Shifts the bits of a number to the left by a specified number of positions.
- **Right Shift (`>>`):** Shifts the bits of a number to the right by a specified number of positions.

These operators form the foundation for our binary manipulation functions. Detailed explanations and examples can be found at [GeeksforGeeks Bitwise Operators](https://www.geeksforgeeks.org/python-bitwise-operators/).

In summary, Task 1 involves implementing critical binary operations using circular shifts and bitwise operators. These functions serve as building blocks for more complex algorithms like SHA-256 and are widely applicable in many areas of computer science. By understanding and implementing `rotl`, `rotr`, `ch`, and `maj`, you not only gain proficiency in bitwise manipulation but also lay the groundwork for further exploration into cryptographic and low-level programming concepts.

For further reading and a deeper understanding, explore the [Circular Shift](https://en.wikipedia.org/wiki/Circular_shift) and [Bitwise Operators](https://www.geeksforgeeks.org/python-bitwise-operators/) articles.


### 1. [rotl(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotl)
   - **Purpose:** Rotates the bits of a 32-bit unsigned integer `x` to the left by `n` positions.
   - **Method:** Combine a left shift and a right shift using bitwise OR, ensuring the wrapped-around bits are correctly placed.
   - **Considerations:** Use a bitmask (`0xFFFFFFFF`) to maintain 32-bit constraints, and handle cases where `n >= 32` by reducing `n` modulo 32.

In [1]:
def rotl(x, n=1):
    # How many bits we need to represent x (at least 1)
    width = x.bit_length() or 1
    mask = (1 << width) - 1 # For example if width is 7, mask is 0b1111111

    # Make sure n is in the range [0, width]
    n %= width

    # Do the rotation:
    # - shift x to the left by n bits
    # - bitwise AND with the mask to keep only the width least significant bits
    # - bitwise OR with the bits that were "shifted out" to the left
    return ((x << n) & mask) | (x >> (width - n))

# Example
x = 0b1010101 # 85
rotated = rotl(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))


Original: 1010101
Rotated:  0101011


### 2. [rotr(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotr)
   - **Purpose:** Rotates the bits of a 32-bit unsigned integer `x` to the right by `n` positions.
   - **Method:** Similar to `rotl`, but with right and left shifts interchanged.
   - **Considerations:** Maintain 32-bit integrity using a bitmask and modulo operations on `n`.

In [2]:
def rotr(x, n=1):
    width = x.bit_length() or 1 # How many bits we need to represent x (at least 1)
    mask = (1 << width) - 1  # For example if width is 7, mask is 0b1111111
    x &= mask # Make sure x is in the range [0, 2**width)
    n %= width # Make sure n is in the range [0, width)
    return (x >> n) | ((x << (width - n)) & mask) # Do the rotation

# Example
x = 0b0101010 # 42
rotated = rotl(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))

Original: 101010
Rotated:  010101


### 3. [ch(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)
   - **Purpose:** Implements a bitwise "choice" function where for each bit position, if the corresponding bit in `x` is `1`, the output takes the bit from `y`; otherwise, it takes the bit from `z`.
   - **Method:** Utilize bitwise operators to combine the bits from `y` and `z` based on `x`.
   - **Application:** This function is commonly used in cryptographic algorithms to blend bits in a controlled manner.

In [3]:
def ch(x, y, z):
    # (x AND y) XOR (NOT x AND z)
    return (x & y) ^ (~x & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0  
z = 0b1111 # 15

result = ch(x, y, z)
print("Expeted:", bin(y))
print("Result:", bin(result))

Expeted: 0b0
Result: 0b101


### 4. [maj(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)
   - **Purpose:** Computes the majority vote of the bits in `x`, `y`, and `z` for each bit position. The output bit is `1` if at least two of the three corresponding bits are `1`.
   - **Method:** Combine bitwise ANDs and ORs to efficiently determine the majority.
   - **Application:** This function is another staple in cryptographic hash functions, ensuring robust diffusion of input bits.

In [4]:
def maj(x, y, z):
    # (x AND y) XOR (x AND z) XOR (y AND z)
    return (x & y) ^ (x & z) ^ (y & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0
z = 0b1111 # 15

print("Expected:", bin(x))
print("Result:  ", bin(maj(x, y, z)))

Expected: 0b1010
Result:   0b1010


[Back to Top](#table-of-contents)

## **Task 2: Hash Functions**

In this task, we will convert a classic C hash function from *The C Programming Language* by Brian Kernighan and Dennis Ritchie into Python. The original hash function uses the `ord()` function (in Python, this returns the Unicode code point for a given character) to process each character in a string. For further details on the `ord()` function, see the [ord() Function documentation](https://www.w3schools.com/python/ref_func_ord.asp).

### The Original C Hash Function

The C code for the hash function is as follows:

```c
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}


### Explanation:
- **Loop through the string:** The function iterates over each character in the input string `s`.
- **Calculate hash value:** For every character, the hash value is updated by multiplying the current hash value by `31` and then adding the ASCII value of the character. This multiplication factor of ``31`` is chosen because it is a small prime number that helps in spreading out the hash values over a wide range.
- **Modulo Operation:** The final hash value is reduced using a modulo operation with `101`, another prime number, which helps in distributing the hash values more uniformly across available buckets (this can be especially useful for hash tables).

For more details on the concept of hash sizes and the reasoning behind such choices, refer to [ Hash Size in The C Programming Language.](https://www.cimat.mx/ciencia_para_jovenes/bachillerato/libros/%5BKernighan-Ritchie%5DThe_C_Programming_Language.pdf)

## Porting to Python
To implement the hash function in Python:

- **Using [`ord()`](https://www.w3schools.com/python/ref_func_ord.asp):** The `ord()` function is used to obtain the integer representation (Unicode code point) of each character in the string.

- **Iteration over the string:** We can iterate over the string directly since Python strings are iterable.

- **Modulo Operation:** The modulo operation `(% 101)` is applied at the end to get the final hash value.

### **Python Implementation:**

In [5]:
def hash(s, base=31, modulus=101):
    hashValue = 0
    for c in s:
        hashValue = ord(c) + base * hashValue # ord(c) returns the ASCII value of c
    return hashValue % modulus 

In [6]:
# Some test strings
test_strings = [
    "hello", "world", "foo", "bar", "baz", "hash", "collision", "test",
    "abcd", "abce", "abcf", "java", "python", "rust", "c++", "javascript"
]

for string in test_strings:
    print(f"{string!r} -> {hash(string):>08b}, {hash(string)}")

'hello' -> 00010001, 17
'world' -> 00100010, 34
'foo' -> 01000101, 69
'bar' -> 00100100, 36
'baz' -> 00101100, 44
'hash' -> 00001111, 15
'collision' -> 00001000, 8
'test' -> 01010110, 86
'abcd' -> 01100100, 100
'abce' -> 00000000, 0
'abcf' -> 00000001, 1
'java' -> 01011101, 93
'python' -> 01011011, 91
'rust' -> 00010001, 17
'c++' -> 00111100, 60
'javascript' -> 00101011, 43


*Collison in `rust -> 17` and `hello -> 17`*

## Testing the hash(string) function
This code generates unique random alphanumeric strings and evaluates how different hash function parameters (bases and moduli) affect collision rates. It creates a set of unique strings using random selection from letters and digits, then applies the hash function to each string for various (base, modulus) pairs. By tracking when multiple strings yield the same hash value, the script computes collision counts and rates, ultimately printing a summary that highlights the commonly used parameters of 31 and 101.

In [7]:
import string
import random

def generate_unique_strings(count, length=8):
    """Generate a set of unique random alphanumeric strings of given length."""
    unique_strings = set()
    
    while len(unique_strings) < count:
        new_string = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
        unique_strings.add(new_string)
    
    return list(unique_strings)

def test_collisions(strings, bases, moduli):
    """
    strings:  list of test strings to hash
    bases:    list of possible 'base' values
    moduli:   list of possible 'modulus' values
    
    Returns a dictionary with collision statistics for each (base, modulus) pair.
    """
    
    results = {}
    
    for base in bases:
        for mod in moduli:
            # Dictionary to keep track of hash -> list of strings
            hash_map = {}
            
            for s in strings:
                h = hash(s, base, mod)
                if h not in hash_map:
                    hash_map[h] = [s]
                else:
                    hash_map[h].append(s)
            
            # Count collisions by counting the number of strings in each bucket
            collision_count = sum(len(lst) - 1 for lst in hash_map.values() if len(lst) > 1)
            total_strings = len(strings) 
            
            results[(base, mod)] = {
                "collision_count": collision_count,
                "collision_rate": collision_count / total_strings if total_strings else 0,
                "hash_map": hash_map
            }
    
    return results

In [8]:
# Example test strings
test_strings = generate_unique_strings(1000, 8)
    
# Possible bases and moduli to test
candidate_bases = [31, 144, 257]
candidate_moduli = [101, 5_054, 10_007 ] 
    
# Run tests
collision_results = test_collisions(test_strings, candidate_bases, candidate_moduli)
    
# Print summary
for (base, mod), stats in collision_results.items():
    if (base, mod) == (31, 101):
        print("*", end =" ")
    print(f"hash(s,{base},{mod}) -> Collisions: {stats['collision_count']}, "
        f"Collision Rate: {stats['collision_rate']:.2f}")

* hash(s,31,101) -> Collisions: 899, Collision Rate: 0.90
hash(s,31,5054) -> Collisions: 76, Collision Rate: 0.08
hash(s,31,10007) -> Collisions: 54, Collision Rate: 0.05
hash(s,144,101) -> Collisions: 899, Collision Rate: 0.90
hash(s,144,5054) -> Collisions: 92, Collision Rate: 0.09
hash(s,144,10007) -> Collisions: 47, Collision Rate: 0.05
hash(s,257,101) -> Collisions: 899, Collision Rate: 0.90
hash(s,257,5054) -> Collisions: 92, Collision Rate: 0.09
hash(s,257,10007) -> Collisions: 44, Collision Rate: 0.04


### Why `31` and `101`

After testing various candidates—such as bases `[31, 37, 257]` and moduli `[101, 127, 10_007]`—the values `31` and `101` have emerged as effective choices. As highlighted in Joshua Bloch's [*Effective Java*](https://ia800308.us.archive.org/16/items/java_20230528/Joshua%20Bloch%20-%20Effective%20Java%20%283rd%29%20-%202018.pdf):

> The value 31 was chosen because it is an odd prime. If it were even and the multiplication overflowed, information would be lost, because multiplication by 2 is equivalent to shifting. The advantage of using a prime is less clear, but it is traditional.  
> A nice property of 31 is that the multiplication can be replaced by a shift and a subtraction for better performance on some architectures: `31 * i == (i << 5) - i`. Modern VMs do this sort of optimization automatically.

Using `31` as the multiplier helps to generate a well-distributed hash by combining the contributions of each character in a string. Its odd prime nature avoids the pitfalls of even multipliers—such as losing information on overflow—and the computational shortcut (shifting and subtracting) can improve performance.

Similarly, the modulus `101` is chosen because it is a prime number. A prime modulus helps minimize collisions by ensuring a more uniform distribution of hash values. While the exact advantage of using a prime modulus like `101` might be less pronounced than that of the multiplier, it remains a traditional choice that has been proven effective in practice.

In summary, the combination of `31` and `101` has been historically favored in hash function design due to their mathematical properties and performance benefits, which remain relevant even as testing with other numbers (e.g., `37`, `257` for the base and `127`, `10_007` for the modulus) might show similar behavior in certain scenarios.


[Back to Top](#table-of-contents)

## **Task 3: SHA256**
...

[Back to Top](#table-of-contents)

## **Task 4: Prime Numbers**
...

[Back to Top](#table-of-contents)

## **Task 5: Roots**
...

[Back to Top](#table-of-contents)

## **Task 6: Proof of Work**
...

[Back to Top](#table-of-contents)

## **Task 7: Turing Machines**
...

[Back to Top](#table-of-contents)

## **Task 8: Computational Complexity**
...

[Back to Top](#table-of-contents)

## **Conclusion**
...

[Back to Top](#table-of-contents)